In [1]:
# Identify athletes meeting 2025 Taiwan Open selection rules

#1. Singapore citizen
#2. No athlete younger than 16 years of age on 31 December 2025 may be entered or permitted to compete.
#3. Athletes may be required to compete at any identified competitions for fitness and performance monitoring at least 6 months before competition dates. These will include SA organized competitions or any other sanctioned competitions.
#4. No road races results will be accepted


%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic

from google.cloud import storage



In [629]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [630]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,28/12/02,IND,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,9/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,6/9/10,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,20/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local


In [631]:
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.iloc[rowIndex,16]
    year = competitors.iloc[rowIndex,17]    
    
    if 'to' in date or ' - ' in date:
        
        pos = re.search('to|\s\-\s', date)
                                                
        # Splice string to day and month

        split_pos_start=pos.start()+3

            
        final_date = date[split_pos_start:] # left string post splicing
        final_year = year[2:]
        
        event_date = final_date + '/' + final_year
        
        competitors.loc[rowIndex, 'event_date'] = event_date
        
    elif re.search('\w\-\w', date):
        
        if competitors.iloc[rowIndex, 15] == "National School Games":
        
            event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd
        
            competitors.loc[rowIndex, 'event_date'] = event_date
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        
        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       
    

In [632]:
# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], errors='coerce')

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92146/2667859050.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], errors='coerce')


In [633]:
# Choose last 6 months or 12 months of results 

#competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

competitors=competitors.reset_index(drop=True)


In [634]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')


competitors.to_csv('athletes_time_filtered.csv', sep=',', encoding='utf-8-sig', index=False)

In [635]:
competitors[competitors['NAME']=='SOH RUI YONG GUILLAUME']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv
658,SOH RUI YONG GUILLAUME,04:59.3,INDIVIDUAL,33,12,None,1 mile,None,None,None,...,Mid,Male,Vitaly West Mininster Mile 5K (UK),22-Sep,2024,International,22-Sep-24,2024-09-22,214 days 18:31:06.064614,214.0
660,SOH RUI YONG GUILLAUME,31:11.4,INDIVIDUAL,33,133,None,10000m,None,None,None,...,Long,Male,"Tokai University Long Distance Challenge 10,0...",25-Nov,2024,International,25-Nov-24,2024-11-25,150 days 18:31:06.064614,150.0
661,SOH RUI YONG GUILLAUME,15:07.7,INDIVIDUAL,33,46,None,5000m,None,None,None,...,Long,Male,Friday Night Under the Lights 5K (UK),13-Sep,2024,International,13-Sep-24,2024-09-13,223 days 18:31:06.064614,223.0
662,SOH RUI YONG GUILLAUME,08:35.8,INDIVIDUAL,33,None,None,3000m,None,None,None,...,Long,Male,Hercules Wimbledon Athletics Club 3000m,14-Jun,2024,International,14-Jun-24,2024-06-14,314 days 18:31:06.064614,314.0


In [636]:
mask = (competitors['DIVISION'].str.contains(r'B', na=False) & competitors['COMPETITION'].str.contains(r'National School Games', na=False))
competitors.loc[mask, 'AGE'] = '16'

In [637]:
competitors['DOB_new'] = pd.to_datetime(competitors['DOB'], errors='coerce')

competitors['year_extract']=competitors['DOB_new'].dt.strftime('%Y')

competitors['year_extract']=pd.to_numeric(competitors['year_extract'])

competitors['age_extract'] = 2025 - competitors['year_extract']


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92146/3522909459.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  competitors['DOB_new'] = pd.to_datetime(competitors['DOB'], errors='coerce')


In [638]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


competitors['age_extract'] = competitors['age_extract'].apply(age)


In [639]:
# If NSG event then choose AGE otherwise choose age_extract

condition1=competitors['COMPETITION']=='National School Games'


competitors['age_extract']=competitors['AGE'].where((condition1), competitors['age_extract'].values)


In [640]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,DOB_new,year_extract,age_extract
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,11/9 to 11/10,2024,Local,11/10/24,2024-11-10,165 days 18:31:06.064614,165.0,1990-11-24,1990.0,35.0
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,11/9 to 11/10,2024,Local,11/10/24,2024-11-10,165 days 18:31:06.064614,165.0,1995-04-05,1995.0,30.0
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,9/7 to 9/7,2024,Local,9/7/24,2024-09-07,229 days 18:31:06.064614,229.0,1997-04-23,1997.0,28.0
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,6/22 to 6/22,2024,Local,6/22/24,2024-06-22,306 days 18:31:06.064614,306.0,2019-01-08,2019.0,6.0
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,6/22 to 6/22,2024,Local,6/22/24,2024-06-22,306 days 18:31:06.064614,306.0,2074-03-03,2074.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,53 days 18:31:06.064614,53.0,2002-12-28,2002.0,23.0
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,53 days 18:31:06.064614,53.0,2009-09-07,2009.0,16.0
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,53 days 18:31:06.064614,53.0,2010-06-09,2010.0,15.0
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,2/23 to 3/2,2025,Local,3/2/25,2025-03-02,53 days 18:31:06.064614,53.0,2009-07-20,2009.0,16.0


In [641]:
competitors['AGE'] = competitors['AGE'].replace(regex=r'-', value=np.NaN)
competitors['AGE'] = competitors['AGE'].replace(regex=r'nan', value=np.NaN)


competitors['AGE'] = pd.to_numeric(competitors['AGE'])

#athletes_selected = competitors[((competitors['AGE']>=15) & (competitors['AGE']<=17))]

#athletes_selected = competitors[(competitors['YEAR']=='2024')]

In [642]:
# Converts any time format into seconds plus corrects time format errors for 10,000m and 1500m

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

                    
                elif '1500m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                    
                    if len(substring)==7:  # format is X:XX:XX and not XX:XX:XX 
                        
                        idx = 4 # 5th character position
                        replacement = "."
                        metric = '0' + metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                    
                        
                    else:  # format is XX:XX:XX
                        
                        idx = 5 # 5th character position
                        replacement = "."
                        metric = metric[:idx] + replacement + metric[idx+1:]                
                
                        m,s = metric.split(':')            

                        output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())  
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [643]:
athletes=competitors

In [644]:
'''
# OLD MAPPING

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'80', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'^600$', na=True, regex=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'^3000m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles



mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles|Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles(0.838m)'



mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))  # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False) & athletes['EVENT_CLASS'].str.contains('None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles(1.067m)'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'



mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'


mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False)  & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.762m)'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'



mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False)  & athletes['REGION'].str.contains('International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles(0.914m)'



# Throws


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(600g)'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['DIVISION'].str.contains(r'Open|OPEN', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) #NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(600g)'


mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(800g)'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['DIVISION'].str.contains(r'Open|OPEN', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw(800g)'



mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['DIVISION'].str.contains(r'Open', na=False)))# NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['DIVISION'].str.contains(r'Open', na=False)))# NEW
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & (athletes['GENDER']=='Male'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))& (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(4kg)'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))& (athletes['GENDER']=='Male'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put(7.26kg)'



mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(4kg)'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False, regex=True) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(7.26kg)'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))& (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(4kg)'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))& (athletes['GENDER']=='Male'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw(7.26kg)'



mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['EVENT_CLASS'].str.contains(r'2', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(2kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.00kg|1kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(1kg)'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False)) # New
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(2kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw|Discus', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False)) # New
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(1kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(1kg)'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw(2kg)'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump|High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$|^Long\sjump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump|Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault|Pole vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'3000m steeplechase', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.762m)'
mask = (athletes['EVENT'].str.contains(r'3000m steeplechase', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.914m)'

mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False)  & athletes['DISTANCE'].str.contains(r'3000', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.762m)'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False)  & athletes['DISTANCE'].str.contains(r'3000', na=False) & athletes['GENDER'].str.contains(r'Male', na=False) & athletes['DIVISION'].str.contains(r'Open', na=False)) # NEW
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase(0.914m)'


#mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'5000m Race Walk', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'5000', na=False) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'

#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

#mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'


'''

"\n# OLD MAPPING\n\nfor col in athletes.columns:\n    athletes[col] = athletes[col].astype(str)\n    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)\n    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)\n    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)\n    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)\n    athletes[col] = athletes[col].str.strip()\n\n# Create temporary mapped event column\n\nathletes['MAPPED_EVENT']=''\n\n# Correct javelin category\n\nmask = athletes['EVENT'].str.contains(r'Javelin', na=True)\nathletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'\n\n\n# Running\n\n#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))\n#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'\n#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))\n#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'\n#mask = (athl

In [645]:
# NEW MAPPING
# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()


# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open|Invitational', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('None|0.762|0.914', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='4kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7.26', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'7.26kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'4.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'2kg|2.00kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



In [646]:
# Convert results and seed into seconds format

athletes.reset_index(drop=True, inplace=True)

for col in athletes.columns:
    
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()


for i in range(len(athletes)):
    
    result_out=''
        
    rowIndex = athletes.index[i]

    input_string=athletes.iloc[rowIndex,6]    # event description
    
    metric=athletes.iloc[rowIndex,1] # result
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric=='DNS' or metric=='NH' or metric=='NM' or metric=='FOUL' or metric=='DNF' or metric=='SR' :
        continue
    
    result_out = convert_time(i, input_string, metric)
    print('line', i, input_string, metric, result_out)

    
         
    athletes.loc[rowIndex, 'RESULT_CONV'] = result_out


line 0 Hammer Throw 32.40m 32.4
line 1 Pole Vault 4.41m 4.41
line 2 Run 07:38.0 458.0
line 3 Run 15.03 15.03
line 4 Javelin Throw 22.43m 22.43
line 5 Shot Put 8.56m 8.56
line 6 Discus Throw 22.96m 22.96
line 7 Shot Put 8.14m 8.14
line 8 Run 07:00.5 420.5
line 9 Race Walk 28:09.6 1689.6
line 10 Dash 48.68 48.68
line 11 Triple Jump 6.41m 6.41
line 12 Dash 17.32 17.32
line 13 Long Jump 2.17m 2.17
line 14 Run 02:10.7 130.7
line 15 Dash 11.98 11.98
line 16 Run 02:27.7 147.7
line 17 Dash 59.82 59.82
line 18 Dash 14.11 14.11
line 19 Dash 25.38 25.38
line 20 Dash 12.02 12.02
line 21 Run 02:12.8 132.8
line 22 Run 04:23.9 263.9
line 23 Run 02:20.0 140.0
line 24 Dash 17.18 17.18
line 25 Dash 01:23.7 83.7
line 26 Dash 13.82 13.82
line 27 Dash 15.06 15.06
line 28 Dash 01:03.8 63.8
line 29 Dash 31.98 31.98
line 30 Dash 12.59 12.59
line 31 Dash 01:00.0 60.0
line 32 High Jump 1.25m 1.25
line 33 Dash 13.04 13.04
line 34 Race Walk 29:15.4 1755.4
line 35 Run 02:16.0 136.0
line 36 Run 04:57.0 297.0
line 3

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_92146/4168403502.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  athletes.loc[rowIndex, 'RESULT_CONV'] = result_out


line 1445 200m 22.52 22.52
line 1446 400m 49.75 49.75
line 1447 100m 10.85 10.85
line 1448 100m 10.81 10.81
line 1449 100m 11.19 11.19
line 1450 200m 22.33 22.33
line 1451 100m 11.2 11.2
line 1452 200m 22.32 22.32
line 1453 100m 10.91 10.91
line 1454 200m 22.66 22.66
line 1455 100m 10.91 10.91
line 1456 100m 11.29 11.29
line 1457 100m 11.31 11.31
line 1458 400m 48.7 48.7
line 1459 100m 11.47 11.47
line 1460 100m 11.56 11.56
line 1461 200m 23.17 23.17
line 1462 100m 11.77 11.77
line 1463 100m 11.64 11.64
line 1464 100m 10.47 10.47
line 1465 110m Hurdles 14.2 14.2
line 1466 800m 02:14.4 134.4
line 1467 Long Jump 5.82 5.82
line 1468 Triple Jump 12.77 12.77
line 1469 400m 59.69 59.69
line 1470 Long Jump 6.37 6.37
line 1471 60m 7.3 7.3
line 1472 Long Jump 5.8 5.8
line 1473 Triple Jump 12.79 12.79
line 1474 200m 21.7 21.7
line 1475 60m 6.84 6.84
line 1476 200m 21.68 21.68
line 1477 60m 6.97 6.97
line 1478 Long Jump 5.88 5.88
line 1479 Triple Jump 13.01 13.01
line 1480 100m 11.62 11.62
line 1

line 3203 200m 00:27.85 27.85
line 3204 200m 00:29.38 29.38
line 3205 400m Hurdles 01:24.33 84.33
line 3206 3000m 13:00.43 780.43
line 3207 Triple Jump 9.87 9.87
line 3208 3000m 12:47.4 767.4
line 3209 400m 01:11.1 71.1
line 3210 400m 01:04.9 64.9
line 3211 400m 01:12.0 72.0
line 3212 100m 00:14.71 14.71
line 3213 100m 00:13.72 13.72
line 3214 800m 02:54.90 174.9
line 3215 200m 00:29.56 29.56
line 3216 200m 00:29.43 29.43
line 3217 200m 00:28.20 28.2
line 3218 Long Jump 4.35 4.35
line 3219 Long Jump 5.61 5.61
line 3220 3000m S/C 12:52.26 772.26
line 3221 400m 01:03.66 63.66
line 3222 400m Hurdles 01:22.68 82.68
line 3223 200m 00:28.06 28.06
line 3224 4x400m Relay 04:36.5 276.5
line 3225 4x400m Relay 04:04.2 244.2
line 3226 3000m Race Walk 22:25.53 1345.53
line 3227 3000m Race Walk 22:28.50 1348.5
line 3228 1500m 05:37.17 337.17
line 3229 4x100m Relay 00:52.68 52.68
line 3230 4x400m Relay 04:34.92 274.92
line 3231 100m 00:18.68 18.68
line 3232 100m 00:18.84 18.84
line 3233 100m 00:18.16

line 4679 100m 00:14.18 14.18
line 4680 100m 00:13.68 13.68
line 4681 100m 00:14.19 14.19
line 4682 3000m 13:07.88 787.88
line 4683 3000m 13:55.69 835.69
line 4684 3000m 12:39.98 759.98
line 4685 Javelin 28.04 28.04
line 4686 Javelin 25.36 25.36
line 4687 Javelin 25.69 25.69
line 4688 Triple Jump 9.29 9.29
line 4689 Triple Jump 10.10 10.1
line 4690 800m 02:41.58 161.58
line 4691 800m 02:35.29 155.29
line 4692 800m 02:52.12 172.12
line 4693 200m 00:29.92 29.92
line 4694 200m 00:29.18 29.18
line 4695 200m 00:29.92 29.92
line 4696 200m 00:27.69 27.69
line 4697 200m 00:28.90 28.9
line 4698 Triple Jump 9.47 9.47
line 4699 Triple Jump 10.44 10.44
line 4700 Triple Jump 9.56 9.56
line 4701 100m Hurdles 00:16.79 16.79
line 4702 100m Hurdles 00:17.16 17.16
line 4703 80m Hurdles 00:14.03 14.03
line 4704 80m Hurdles 00:13.36 13.36
line 4705 80m Hurdles 00:14.61 14.61
line 4706 3000m 12:52.40 772.4
line 4707 3000m 12:33.01 753.01
line 4708 3000m 12:37.10 757.1
line 4709 3000m 13:18.21 798.21
line 4

...

line 19760 Dash 12.33 12.33
line 19761 Dash 12.78 12.78
line 19762 Dash 12.53 12.53
line 19763 Dash 12.77 12.77
line 19764 Dash 13.02 13.02
line 19765 Dash 11.37 11.37
line 19766 Dash 12.27 12.27
line 19767 Dash 12.39 12.39
line 19768 Dash 12.44 12.44
line 19769 Dash 12.41 12.41
line 19770 Dash 13.16 13.16
line 19771 Dash 11.61 11.61
line 19772 Dash 13.58 13.58
line 19773 Dash 12.03 12.03
line 19774 Dash 11.86 11.86
line 19775 Dash 12.04 12.04
line 19776 Dash 12.35 12.35
line 19777 Dash 12.08 12.08
line 19778 Dash 12.46 12.46
line 19779 Dash 11.75 11.75
line 19780 Dash 12.46 12.46
line 19781 Dash 13.27 13.27
line 19782 Dash 16.12 16.12
line 19783 Dash 11.48 11.48
line 19784 Dash 12.37 12.37
line 19785 Dash 11.98 11.98
line 19786 Dash 11.88 11.88
line 19787 Dash 12.26 12.26
line 19788 Dash 12.16 12.16
line 19789 Dash 11.79 11.79
line 19790 Dash 11.35 11.35
line 19791 Dash 12.11 12.11
line 19792 Dash 12.04 12.04
line 19793 Dash 11.52 11.52
line 19794 Dash 12.2 12.2
line 19795 Dash 11.28 

line 21262 Dash 57.98 57.98
line 21263 Dash 55.88 55.88
line 21264 Dash 54.5 54.5
line 21265 Dash 54.87 54.87
line 21266 Dash 57.83 57.83
line 21267 Dash 59.89 59.89
line 21268 Dash 51.73 51.73
line 21269 Dash 52.21 52.21
line 21270 Dash 57 57.0
line 21271 Dash 54.8 54.8
line 21272 Dash 01:00.9 60.9
line 21273 Dash 53.87 53.87
line 21274 Dash 53.83 53.83
line 21275 Dash 52.58 52.58
line 21276 Dash 53.48 53.48
line 21277 Dash 53.95 53.95
line 21278 Dash 53.7 53.7
line 21279 Dash 52.82 52.82
line 21280 Dash 53.39 53.39
line 21281 Dash 52.85 52.85
line 21282 Dash 53.43 53.43
line 21283 Dash 53.11 53.11
line 21284 Dash 53.57 53.57
line 21285 Dash 57.31 57.31
line 21286 Dash 57.25 57.25
line 21287 Dash 52.51 52.51
line 21288 Dash 53.11 53.11
line 21289 Run 05:26.4 326.4
line 21290 Run 04:56.0 296.0
line 21291 Run 05:17.1 317.1
line 21292 Run 04:57.8 297.8
line 21293 Run 05:55.4 355.4
line 21294 Run 04:47.1 287.1
line 21295 Run 05:00.4 300.4
line 21296 Run 05:21.7 321.7
line 21297 Run 05:23.

line 22802 Dash 12.3 12.3
line 22803 Dash 11.6 11.6
line 22804 Dash 13.52 13.52
line 22805 Dash 11.76 11.76
line 22806 Dash 12.66 12.66
line 22807 Dash 11.68 11.68
line 22808 Dash 13.25 13.25
line 22809 Dash 13.81 13.81
line 22810 Dash 14.54 14.54
line 22811 Long Jump 1.40m 1.4
line 22812 Long Jump 1.05m 1.05
line 22813 Long Jump 1.44m 1.44
line 22814 Long Jump 1.22m 1.22
line 22815 Long Jump 1.04m 1.04
line 22816 Long Jump 1.37m 1.37
line 22817 Long Jump 1.26m 1.26
line 22818 Long Jump 1.38m 1.38
line 22819 Long Jump 1.29m 1.29
line 22820 Long Jump 1.43m 1.43
line 22821 Long Jump 1.58m 1.58
line 22822 Long Jump 1.32m 1.32
line 22823 Long Jump 1.33m 1.33
line 22824 Long Jump 1.59m 1.59
line 22825 Long Jump 1.30m 1.3
line 22826 Long Jump 1.16m 1.16
line 22827 Long Jump 1.38m 1.38
line 22828 Long Jump 1.48m 1.48
line 22829 Long Jump 1.61m 1.61
line 22830 Long Jump 1.60m 1.6
line 22831 Long Jump 1.36m 1.36
line 22832 Long Jump 1.58m 1.58
line 22833 Long Jump 1.24m 1.24
line 22834 Long Jum

line 24237 Relay 35.22 35.22
line 24238 Relay 29.82 29.82
line 24239 Relay 37.74 37.74
line 24240 Relay 31.19 31.19
line 24241 Relay 35.67 35.67
line 24242 Relay 30.43 30.43
line 24243 Relay 34.11 34.11
line 24244 Relay 32.2 32.2
line 24245 Relay 30.02 30.02
line 24246 Relay 33.63 33.63
line 24247 Relay 30.14 30.14
line 24248 Relay 31.82 31.82
line 24249 Relay 33.13 33.13
line 24250 Relay 35.11 35.11
line 24251 Relay 31.06 31.06
line 24252 Relay 30.43 30.43
line 24253 Relay 36.68 36.68
line 24254 Relay 39.65 39.65
line 24255 Relay 37.39 37.39
line 24256 Relay 33.07 33.07
line 24257 Relay 33.45 33.45
line 24258 Relay 31.43 31.43
line 24259 Relay 32.58 32.58
line 24260 Relay 30.33 30.33
line 24261 Relay 30.45 30.45
line 24262 Relay 32.65 32.65
line 24263 Relay 38.57 38.57
line 24264 Relay 34.33 34.33
line 24265 Relay 36.55 36.55
line 24266 Relay 37.25 37.25
line 24267 Relay 34.68 34.68
line 24268 Relay 37.16 37.16
line 24269 Relay 37.28 37.28
line 24270 Relay 39.54 39.54
line 24271 Relay

line 25769 Hurdles 33.34 33.34
line 25770 Hurdles 33.21 33.21
line 25771 Hurdles 33.72 33.72
line 25772 Hurdles 35.28 35.28
line 25773 Hurdles 35.62 35.62
line 25774 Hurdles 38.62 38.62
line 25775 Hurdles 32.33 32.33
line 25776 Hurdles 34.58 34.58
line 25777 Hurdles 35 35.0
line 25778 Hurdles 31.39 31.39
line 25779 Hurdles 33.86 33.86
line 25780 Hurdles 32.75 32.75
line 25781 Hurdles 32.34 32.34
line 25782 Hurdles 37.98 37.98
line 25783 Hurdles 37.21 37.21
line 25784 Dash 22.38 22.38
line 25785 Dash 23.72 23.72
line 25786 Dash 23.67 23.67
line 25787 Dash 23.21 23.21
line 25788 Dash 23.62 23.62
line 25789 Dash 24.62 24.62
line 25790 Dash 26.77 26.77
line 25791 Dash 35.75 35.75
line 25792 Dash 21.96 21.96
line 25793 Dash 24.53 24.53
line 25794 Dash 22.29 22.29
line 25795 Dash 22.31 22.31
line 25796 Dash 22.77 22.77
line 25797 Dash 23.8 23.8
line 25798 Dash 23.47 23.47
line 25799 Dash 22.88 22.88
line 25800 Dash 23.47 23.47
line 25801 Dash 23.4 23.4
line 25802 Dash 22.71 22.71
line 25803 

In [647]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')

athletes.to_csv("athletes_time_converted.csv", encoding='utf-8')

In [602]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')

benchmarks = pd.read_csv("2025 Taiwan Open Benchmarks.csv")


In [603]:
benchmarks

,EVENT,GENDER,BENCHMARK
0,100m,Male,10.68
1,100m,Female,12.2
2,200m,Male,21.8
3,200m,Female,26.14
4,400m,Male,49.56
5,400m,Female,57.48
6,800m,Male,01:58.7
7,800m,Female,02:19.0
8,1500m,Male,04:07.7
9,1500m,Female,05:01.1


In [604]:
for col in benchmarks.columns:
    benchmarks[col] = benchmarks[col].astype(str)
    benchmarks[col] = benchmarks[col].str.replace('\xa0', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\r', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\n', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.strip()


In [605]:
#SEAG_filtered=SEAG[SEAG['HEAT'].isnull() & SEAG['SUB_EVENT'].isnull()]

In [606]:
# Convert time for benchmarks and results

for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string = benchmarks.iloc[rowIndex,0]
    
    metric=benchmarks.iloc[rowIndex,2]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

0 100m 10.68
1 100m 12.2
2 200m 21.8
3 200m 26.14
4 400m 49.56
5 400m 57.48
6 800m 118.7
7 800m 139.0
8 1500m 247.7
9 1500m 301.1
10 5000m 952.4
11 5000m 1185.0
12 3000m Steeplechase 607.0
13 3000m Steeplechase 755.0
14 110m Hurdles 14.5
15 100m Hurdles 14.5
16 400m Hurdles 53.0
17 400m Hurdles 68.04
18 High Jump 2.09
19 High Jump 1.68
20 Long Jump 7.3
21 Long Jump 5.7
22 Triple Jump 14.7
23 Triple Jump 11.65
24 Shot Put 14.38
25 Shot Put 12.3
26 Discus Throw 43.5
27 Discus Throw 38.5
28 Javelin Throw 64.0
29 Javelin Throw 45.15
30 Hammer Throw 50.0
31 Hammer Throw 50.0
32 Pole Vault 4.8
33 Pole Vault 3.3


In [607]:
benchmarks

,EVENT,GENDER,BENCHMARK,Metric
0,100m,Male,10.68,10.68
1,100m,Female,12.2,12.20
2,200m,Male,21.8,21.80
3,200m,Female,26.14,26.14
4,400m,Male,49.56,49.56
5,400m,Female,57.48,57.48
6,800m,Male,01:58.7,118.70
7,800m,Female,02:19.0,139.00
8,1500m,Male,04:07.7,247.70
9,1500m,Female,05:01.1,301.10


In [608]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put', na=True)

benchmarks.loc[mask, '2%']=benchmarks['Metric']*0.98
benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
benchmarks.loc[mask, '3.5%']=benchmarks['Metric']*0.975
benchmarks.loc[~mask, '3.5%']=benchmarks['Metric']*1.035
benchmarks.loc[mask, '5%']=benchmarks['Metric']*0.95
benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [609]:
benchmarks

,EVENT,GENDER,BENCHMARK,Metric,2%,3.5%,5%
0,100m,Male,10.68,10.68,10.8936,11.05380,11.2140
1,100m,Female,12.2,12.20,12.4440,12.62700,12.8100
2,200m,Male,21.8,21.80,22.2360,22.56300,22.8900
3,200m,Female,26.14,26.14,26.6628,27.05490,27.4470
4,400m,Male,49.56,49.56,50.5512,51.29460,52.0380
5,400m,Female,57.48,57.48,58.6296,59.49180,60.3540
6,800m,Male,01:58.7,118.70,121.0740,122.85450,124.6350
7,800m,Female,02:19.0,139.00,141.7800,143.86500,145.9500
8,1500m,Male,04:07.7,247.70,252.6540,256.36950,260.0850
9,1500m,Female,05:01.1,301.10,307.1220,311.63850,316.1550


In [610]:
'''
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'110m Hurdles(0.914m)', value='110m Hurdles')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'110m Hurdles(0.762m)', value='110m Hurdles')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'400m Hurdles(0.840m)', value='400m Hurdles')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'400m Hurdles(0.762m)', value='400m Hurdles')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Javelin Throw(700g)', value='Javelin Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Javelin Throw(500g)', value='Javelin Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Hammer Throw(5kg)', value='Hammer Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Hammer Throw(3kg)', value='Hammer Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Shot Put(5kg)', value='Shot Put')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Shot Put(3kg)', value='Shot Put')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Discus Throw(1.5kg)', value='Discus Throw')
benchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Discus Throw(1kg)', value='Discus Throw')
'''

"\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'110m Hurdles(0.914m)', value='110m Hurdles')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'110m Hurdles(0.762m)', value='110m Hurdles')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'400m Hurdles(0.840m)', value='400m Hurdles')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'400m Hurdles(0.762m)', value='400m Hurdles')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Javelin Throw(700g)', value='Javelin Throw')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Javelin Throw(500g)', value='Javelin Throw')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Hammer Throw(5kg)', value='Hammer Throw')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Hammer Throw(3kg)', value='Hammer Throw')\nbenchmarks['MAPPED_EVENT'] = benchmarks['EVENT'].replace(regex=r'Shot Put(5kg)', value='Shot Put')\nbenchmarks['MAP

In [611]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [612]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['EVENT', 'GENDER'],
)

In [613]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,year_extract,age_extract,MAPPED_EVENT,RESULT_CONV,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34.0,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,1990.0,35.0,Hammer Throw,32.4,Hammer Throw,50,50.0,49.000,48.75,47.50
1,"Koh, Wei Shien",4.41m,NUS,29.0,3,Open,Pole Vault,0,,L435B06,...,1995.0,30.0,Pole Vault,4.41,Pole Vault,4.8,4.8,4.704,4.68,4.56
2,"Wen Qiang, Koh",07:38.0,SIT,27.0,4,Tertiary Men,Run,2400,,,...,1997.0,28.0,,458.0,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4.0,7,5-Jun,Run,60,,nan,...,2019.0,6.0,,15.03,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50.0,5,Masters,Javelin Throw,0,,nan,...,2074.0,51.0,,22.43,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23.0,2,Open,Relay,400,Trial,None,...,2002.0,23.0,4 x 100m,42.06,NaN,NaN,NaN,NaN,NaN,NaN
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16.0,2,Open,Relay,400,Trial,None,...,2009.0,16.0,4 x 100m,42.06,NaN,NaN,NaN,NaN,NaN,NaN
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15.0,2,Open,Relay,400,Trial,None,...,2010.0,15.0,4 x 100m,42.06,NaN,NaN,NaN,NaN,NaN,NaN
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16.0,2,Open,Relay,400,Trial,None,...,2009.0,16.0,4 x 100m,42.06,NaN,NaN,NaN,NaN,NaN,NaN


In [614]:
df[df['NAME']=='Reuben Lee Siong En']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,year_extract,age_extract,MAPPED_EVENT,RESULT_CONV,EVENT_y,BENCHMARK,Metric,2%,3.5%,5%
1401,Reuben Lee Siong En,47.81,None,nan,6,None,400m,None,None,None,...,2002.0,23.0,400m,47.81,400m,49.56,49.56,50.5512,51.2946,52.038
1529,Reuben Lee Siong En,11,None,nan,None,None,100m,None,None,None,...,2002.0,23.0,100m,11.0,100m,10.68,10.68,10.8936,11.0538,11.214
1570,Reuben Lee Siong En,10.86,None,nan,5,None,100m,None,None,None,...,2002.0,23.0,100m,10.86,100m,10.68,10.68,10.8936,11.0538,11.214
1595,Reuben Lee Siong En,10.9,None,nan,8,None,100m,None,None,None,...,2002.0,23.0,100m,10.9,100m,10.68,10.68,10.8936,11.0538,11.214
1596,Reuben Lee Siong En,22.04,None,nan,3,None,200m,None,None,None,...,2002.0,23.0,200m,22.04,200m,21.8,21.80,22.2360,22.5630,22.890
1661,Reuben Lee Siong En,48.64,None,nan,1,None,400m,None,None,None,...,2002.0,23.0,400m,48.64,400m,49.56,49.56,50.5512,51.2946,52.038
1662,Reuben Lee Siong En,49.79,None,nan,1,None,400m,None,None,None,...,2002.0,23.0,400m,49.79,400m,49.56,49.56,50.5512,51.2946,52.038
1691,Reuben Lee Siong En,21.59,None,nan,6,None,200m,None,None,None,...,2002.0,23.0,200m,21.59,200m,21.8,21.80,22.2360,22.5630,22.890
1692,Reuben Lee Siong En,10.77,None,nan,1,None,100m,None,None,None,...,2002.0,23.0,100m,10.77,100m,10.68,10.68,10.8936,11.0538,11.214
20916,Reuben Lee Siong En,50.78,None,nan,1,None,400m,None,None,None,...,2002.0,23.0,400m,50.78,400m,49.56,49.56,50.5512,51.2946,52.038


In [615]:
# Change to numeric

df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'-', value=np.NaN)
df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'-', value=np.NaN)
df['RESULT_CONV'] = df['RESULT_CONV'].replace(regex=r'nan', value=np.NaN)

df['age_extract'] = df['age_extract'].replace(regex=r'-', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'-', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'nan', value=np.NaN)
df['age_extract'] = df['age_extract'].replace(regex=r'None', value=np.NaN)


df[['2%', '3.5%', '5%', 'RESULT_CONV']] = df[['2%', '3.5%', '5%', 'RESULT_CONV']].apply(pd.to_numeric)


In [616]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_CONV']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_CONV']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_CONV']-df['5%']

df.loc[mask, 'Delta_Benchmark'] = df['RESULT_CONV']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_CONV']
df.loc[~mask, 'Delta3.5'] =  df['3.5%'] - df['RESULT_CONV']
df.loc[~mask, 'Delta5'] =  df['5%'] - df['RESULT_CONV']

df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_CONV']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']


In [617]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta_Benchmark']/df['Metric']*100

In [618]:
#df['TIER'] = np.where((df['Delta_Benchmark']>=0), 'Tier 1',    
#            np.where(((df['Delta_Benchmark']<0) & (df['Delta2']>=0)), 'Tier2', ' '))

df['TIER'] = np.where((df['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((df['Delta_Benchmark']<0) & (df['Delta2']>=0)), 'Tier 2',
                                np.where(((df['Delta2']<0) & (df['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((df['Delta3.5']<0) & (df['Delta5']>=0)), 'Tier 4', ' '))))


In [623]:
df[df['NAME']=='Reuben Lee Siong En']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,Metric,2%,3.5%,5%,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER


In [619]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')

df_filtered=df[df['TIER']!=' ']

df_filtered.to_csv('taiwan_open_mapped.csv', sep=',', encoding='utf-8-sig', index=False)


In [620]:
#df[['age_extract']] = df[['age_extract']].apply(pd.to_numeric)

#athletes_selected = df[((df['age_extract']>=15) & (df['age_extract']<=17))]


In [621]:
athletes_selected=df

In [622]:
# Read name variations from GCS name lists bucket (Still in beta)


athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\xa0', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\r', '', regex=True)
athletes_selected['NAME'] = athletes_selected['NAME'].str.replace('\n', '', regex=True)
athletes_selected['NAME'] = df['NAME'].str.strip()


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})

# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    athletes_selected['NAME'] = athletes_selected['NAME'].replace(regex=rf"^{row['VARIATION']}$", value=f"{row['NAME']}")


In [624]:
# Choose the best result for each event participated by every athlete

best_selection = athletes_selected.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

athletes=best_selection

In [625]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

final_taiwan_selection = athletes[(athletes['TEAM']!='Malaysia')&(athletes['TEAM']!='THAILAND')&(athletes['TEAM']!='China') 
                       &(athletes['TEAM']!='South Korea')&(athletes['TEAM']!='Laos') 
                       &(athletes['TEAM']!='Philippines')&(athletes['TEAM']!='Piboonbumpen Thailand') 
                       &(athletes['TEAM']!='Chinese Taipei')&(athletes['TEAM']!='Gurkha Contingent') 
                       &(athletes['TEAM']!='Australia')&(athletes['TEAM']!='Piboonbumpen Thailand') 
                       &(athletes['TEAM']!='Hong Kong')&(athletes['TEAM']!='PERAK')&(athletes['TEAM']!='Sri Lanka') 
                       &(athletes['TEAM']!='Indonesia')&(athletes['TEAM']!='THAILAND')&(athletes['TEAM']!='MALAYSIA') 
                       &(athletes['TEAM']!='PHILIPPINES') & (athletes['TEAM']!='SOUTH KOREA')&(athletes['TEAM']!='Waseda') 
                       &(athletes['TEAM']!='LAOS')&(athletes['TEAM']!='CHINESE TAIPEI')
                       &(athletes['TEAM']!='INDIA')&(athletes['TEAM']!='Hong Kong, China')&(athletes['TEAM']!='AIC JAPAN')&(athletes['NATIONALITY']!='GBR')&(athletes['NATIONALITY']!='IND')] 


In [626]:
all_ranking = final_taiwan_selection.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
all_ranking['Rank'] = all_ranking.groupby(['GENDER', 'MAPPED_EVENT', 'TIER']).cumcount() + 1


In [627]:
all_ranking=all_ranking[(all_ranking['TIER']!=' ')]

In [628]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Taiwan Open/')


all_ranking.to_csv('taiwan_open.csv', sep=',', encoding='utf-8-sig', index=False)


# Clean manual load data

In [133]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/SEA Games/')


df_SHA = pd.read_csv("SEA_manual_additions.csv", encoding='latin-1')

In [134]:
df_SHA

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,NaN,NaN,NaN,Agus Prayogo,1.0,NaN,NaN,NaN,2:32:59,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
1,NaN,NaN,NaN,ÃArlan Estobo Arbois,2.0,NaN,NaN,NaN,2:33:27,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
2,NaN,NaN,NaN,Nguyen Thanh Hoang,3.0,NaN,NaN,NaN,2:35:49,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
3,NaN,NaN,NaN,Tan Huong Leong,4.0,NaN,NaN,NaN,2:40:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
4,NaN,NaN,NaN,Vanh Pheara,5.0,NaN,NaN,NaN,2:41:26,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
5,NaN,NaN,NaN,Quoc Luong Trinh,6.0,NaN,NaN,NaN,2:41:36,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
6,NaN,NaN,NaN,Sanchai Namkhet,7.0,NaN,NaN,NaN,2:43:47,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
7,NaN,NaN,NaN,Joanito Fernandes,8.0,NaN,NaN,NaN,2:52:41,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
8,NaN,NaN,NaN,Tony Ah-Thit Payne,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN
9,NaN,NaN,NaN,Yang Piseth,NaN,NaN,NaN,NaN,DNF,NaN,...,https://en.wikipedia.org/wiki/Athletics_at_the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phnom Phen,NaN


In [135]:
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\xa0', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\r', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.replace('\n', '', regex=True)
df_SHA['NAME'] = df_SHA['NAME'].str.strip()


In [136]:
df_SHA.to_csv('SEA_manual_additions.csv', index=False, encoding='utf-8')

# Read a csv of name list variations

In [137]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')

names = pd.read_csv("name_variations.csv")

In [138]:
names

,VARIATION,NAME
0,Harry\sIrfan\sCurra,HARRY IRFAN CURRAN
1,"Curran, Harry Irfan",HARRY IRFAN CURRAN
2,Harry Irfan Curran,HARRY IRFAN CURRAN
3,"Curran, Harry Irfann",HARRY IRFAN CURRAN
4,"Curran,\sHarry",HARRY IRFAN CURRAN
...,...,...
370,Goh Xu'en Hannah,"GOH, XU'EN HANNAH"
371,"Low, Jonathan","Low Jun Jie, Jonathan"
372,"Low, Jun Jie Jonathon","Low Jun Jie, Jonathan"
373,"Low, Jun Jie Jonathan","Low Jun Jie, Jonathan"


In [139]:
lines = [line.strip() for line in open('file.csv')]
for x in lines:
    match=re.search(r'^_.*_$',x)
    if match: print x

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (2082507092.py, line 4)

In [ ]:
df['NAME'] = df['NAME'].replace(regex=r'^Harry\sIrfan\sCurra$', value='HARRY IRFAN CURRAN')


In [ ]:
for index, row in names.iterrows():
    print(row['VARIATION'])
    